In [2]:
# --- import packages ---
import sys 
sys.path.append("../..") 
from source import utils,env,main,test
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
import gc

### Formal Test

In [22]:
# gamma setting
gamma_set = [3,4,5]

# Fix general setting
test_settings = {
    "time_lim":15 * 60,"ite_lim":None,"gap_lim":0.01,
    "num":50,"dim":100,"rho":0.5,"sparsity":10,"SNR":3.5,
    "inc_max_num":10,"inc_max_len":2,"exc_max_num":90,"exc_max_len":3,
    "solver": 'gurobi'}

total_instance = 5

In [28]:
# start testing
# store bad instance
bad_ins = []

# record attributes
all_att = []

# record sol time 
all_data_SSR,all_data_SCG = [],[]

# record finding cuts time
all_find_cuts_SSR,all_find_cuts_SCG = [],[]

# record cuts
all_support_SSR,all_zero_SSR = [],[]
all_inc_SCG, all_exc_SCG = [],[]

for gamma_ in gamma_set:
    print('=====')

    # record attributes for each gamma
    record_att = []

    # record sol time for each gamma
    record_SSR,record_SCG = [],[]

    # record finding cuts time for each gamma
    record_find_cuts_SSR,record_find_cuts_SCG = [],[]

    # record cuts for each gamma
    record_support_SSR, record_zero_SSR = [],[]
    record_inc_SCG, record_exc_SCG = [],[]
    for _ in range(total_instance):
        time_log, cut_log, att_log, test_X, test_Y = test.syn_ins_test(progress_log=True,test_alg=['SCG'], \
                                                            gamma_ = gamma_,**test_settings)

        # record bad instance
        if att_log['SCG status'] != 2:
            bad_ins.append((test_X,test_Y,gamma_,test_settings['sparsity']))
        else:
            pass
        del test_X
        del test_Y
        gc.collect
        
        record_SSR.append(time_log['SSR'])
        record_SCG.append(time_log['SCG'])

        record_att.append(att_log)

        record_find_cuts_SSR.append(time_log['SSR find cuts'])
        record_find_cuts_SCG.append(time_log['SCG find cuts'])

        record_support_SSR.append(cut_log['SSR_support'])
        record_zero_SSR.append(cut_log['SSR_zero'])

        record_inc_SCG.append(cut_log['SCG_inclusive'])
        record_exc_SCG.append(cut_log['SCG_exclusive'])

        # delete the variable
        del time_log
        del cut_log
        del att_log
        gc.collect()
        

    
    all_att.append(record_att)
    
    all_data_SSR.append(np.array(record_SSR))
    all_data_SCG.append(np.array(record_SCG))

    all_find_cuts_SSR.append(np.array(record_find_cuts_SSR))
    all_find_cuts_SCG.append(np.array(record_find_cuts_SCG))

    all_support_SSR.append(record_support_SSR)
    all_zero_SSR.append(record_zero_SSR)

    all_inc_SCG.append(record_inc_SCG)
    all_exc_SCG.append(record_exc_SCG)
    print('One gamma done',gamma_)
    print('=====')

In [6]:
# check the bad instance
check = True
save = True

if check == True:
    print('bad ins num:', len(bad_ins))
    print(bad_ins)
    

if save == True:
    save_dic = {}
    save_dic['X'] = []
    save_dic['Y'] = []
    save_dic['gamma'] = []
    save_dic['sparsity'] = []

    if bad_ins !=[]:
        for i in range(len(bad_ins)):
            ins_tuple = bad_ins[i]
            X,Y,gma,spar = ins_tuple[0],ins_tuple[1],ins_tuple[2],ins_tuple[3]
            save_dic['X'].append(X)
            save_dic['Y'].append(Y)
            save_dic['gamma'].append(gma)
            save_dic['sparsity'].append(spar)
    
        # save bad instance
        file_name = 'k_' + str(test_settings['sparsity']) + '_SNR_'+ str(test_settings['SNR']) + '_bad' + '.npy'
        np.save(file_name,save_dic)

### Plot

In [8]:
# Average total running time

# Plot settings
sns.set_style('whitegrid')
font1 = {'weight' : 'bold', 
         'size'   : 25}
fig = plt.figure(figsize=(15,10))

# Draw Average 
line_SSR, = plt.plot(gamma_set,np.mean(all_data_SSR,axis = 1),linestyle="-",color="red",marker="o",linewidth=3.5,markersize = 13)
line_SCG, = plt.plot(gamma_set,np.mean(all_data_SCG,axis = 1),linestyle="-",color="blue",marker="v",linewidth=3.5,markersize = 13)

# Fill with standard deviation / 25% and 75%
avg = np.mean(all_data_SSR,axis = 1)
# std=np.std(all_data_SSR,axis=1)
# r1 = list(map(lambda x: x[0]-x[1], zip(avg, std)))
# r2 = list(map(lambda x: x[0]+x[1], zip(avg, std)))
# plt.fill_between(gamma_set,r1,r2,color = 'orange',alpha = 0.2)
percent_25 = [np.percentile(all_data_SSR[i],25) for i in range(len(gamma_set))]
percent_75 = [np.percentile(all_data_SSR[i],75) for i in range(len(gamma_set))]
plt.fill_between(gamma_set,percent_25,percent_75,color = 'orange',alpha = 0.2)


avg = np.mean(all_data_SCG,axis = 1)
# std=np.std(all_data_SCG,axis=1)
# r1 = list(map(lambda x: x[0]-x[1], zip(avg, std)))
# r2 = list(map(lambda x: x[0]+x[1], zip(avg, std)))
# plt.fill_between(gamma_set,r1,r2,color = 'green',alpha = 0.2)
percent_25 = [np.percentile(all_data_SCG[i],25) for i in range(len(gamma_set))]
percent_75 = [np.percentile(all_data_SCG[i],75) for i in range(len(gamma_set))]
plt.fill_between(gamma_set,percent_25,percent_75,color = 'green',alpha = 0.2)

ax = plt.gca() 
ax.spines['top'].set_color('black')  
ax.spines['right'].set_color('black')
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)

plt.xticks(fontsize=30,fontweight = 'bold')
plt.yticks(fontsize=30,fontweight = 'bold')
plt.xlabel('gamma',fontsize=35,fontweight = 'bold')
plt.ylabel('time(s)',fontsize=35,labelpad = 20,fontweight = 'bold')
plt.legend([line_SSR,line_SCG],['SSR','SCG'],loc = 'upper right',prop = font1)

In [10]:
# relative time gap
relative_time_gap = []

for i in range(len(gamma_set)):
    SSR_time = all_data_SSR[i]
    SCG_time = all_data_SCG[i]
    relative_time_gap.append((SSR_time - SCG_time) / SSR_time)

# Plot settings
sns.set_style('whitegrid')
font1 = {'weight' : 'bold', 
         'size'   : 25}
fig = plt.figure(figsize=(15,10))

# Draw Average 
line_avg_gap, = plt.plot(gamma_set,np.mean(relative_time_gap,axis = 1),linestyle="-",color="red",marker="o",linewidth=3.5,markersize = 13)

# Fill with standard deviation
avg = np.mean(relative_time_gap,axis = 1)
std=np.std(relative_time_gap,axis=1)
r1 = list(map(lambda x: x[0]-x[1], zip(avg, std)))
r2 = list(map(lambda x: x[0]+x[1], zip(avg, std)))
plt.fill_between(gamma_set,r1,r2,color = 'orange',alpha = 0.2)

ax = plt.gca() 
ax.spines['top'].set_color('black')  
ax.spines['right'].set_color('black')
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)

plt.xticks(fontsize=30,fontweight = 'bold')
plt.yticks(fontsize=30,fontweight = 'bold')
plt.xlabel('gamma',fontsize=35,fontweight = 'bold')
plt.ylabel('relative time gap',fontsize=35,labelpad = 20,fontweight = 'bold')

In [12]:
# Retrieve the objective
SSR_obj_list = []
SCG_obj_list = []

for i in range(len(gamma_set)):
    SSR_obj_gamma = []
    SCG_obj_gamma = []
    for j in range(total_instance):
        SSR_obj_gamma.append(all_att[i][j]['SSR obj'])
        SCG_obj_gamma.append(all_att[i][j]['SCG obj'])

    SSR_obj_list.append(SSR_obj_gamma)
    SCG_obj_list.append(SCG_obj_gamma)

# get the relative objective gap
obj_gap_list = []
for i in range(len(gamma_set)):
    obj_gap_gamma = []
    for j in range(total_instance):
        obj_gap_gamma.append((SSR_obj_list[i][j] - SCG_obj_list[i][j]) / SSR_obj_list[i][j])

    obj_gap_list.append(obj_gap_gamma)

# Plot settings
sns.set_style('whitegrid')
font1 = {'weight' : 'bold', 
         'size'   : 25}
fig = plt.figure(figsize=(15,10))

# Draw Average 
line_avg_gap, = plt.plot(gamma_set,np.mean(obj_gap_list,axis = 1),linestyle="-",color="red",marker="o",linewidth=3.5,markersize = 13)

# # Fill with standard deviation
avg = np.mean(obj_gap_list,axis = 1)
std=np.std(obj_gap_list,axis=1)
r1 = list(map(lambda x: x[0]-x[1], zip(avg, std)))
r2 = list(map(lambda x: x[0]+x[1], zip(avg, std)))
plt.fill_between(gamma_set,r1,r2,color = 'orange',alpha = 0.2)

ax = plt.gca() 
ax.spines['top'].set_color('black')  
ax.spines['right'].set_color('black')
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)

plt.xticks(fontsize=30,fontweight = 'bold')
plt.yticks(fontsize=30,fontweight = 'bold')
plt.xlabel('gamma',fontsize=35,fontweight = 'bold')
plt.ylabel('relative objective gap',fontsize=35,labelpad = 20,fontweight = 'bold')

In [14]:
# Retrieve the MIPGap
SSR_gap_list = []
SCG_gap_list = []

for i in range(len(gamma_set)):
    SSR_gap_gamma = []
    SCG_gap_gamma = []
    for j in range(total_instance):
        SSR_gap_gamma.append(all_att[i][j]['SSR MIPGap'])
        SCG_gap_gamma.append(all_att[i][j]['SCG MIPGap'])

    SSR_gap_list.append(SSR_gap_gamma)
    SCG_gap_list.append(SCG_gap_gamma)

# get the MIPGap difference
ratio_list = []
for i in range(len(gamma_set)):
    ratio_gamma = []
    for j in range(total_instance):
        ratio_gamma.append(SSR_gap_list[i][j] - SCG_gap_list[i][j])
    ratio_list.append(ratio_gamma)

# Plot settings
sns.set_style('whitegrid')
font1 = {'weight' : 'bold', 
         'size'   : 25}
fig = plt.figure(figsize=(15,10))

# Draw Average 
line_avg_gap, = plt.plot(gamma_set,np.mean(ratio_list,axis = 1),linestyle="-",color="red",marker="o",linewidth=3.5,markersize = 13)

# Fill with standard deviation
avg = np.mean(ratio_list,axis = 1)
std=np.std(ratio_list,axis=1)
r1 = list(map(lambda x: x[0]-x[1], zip(avg, std)))
r2 = list(map(lambda x: x[0]+x[1], zip(avg, std)))
plt.fill_between(gamma_set,r1,r2,color = 'orange',alpha = 0.2)

ax = plt.gca() 
ax.spines['top'].set_color('black')  
ax.spines['right'].set_color('black')
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)

plt.xticks(fontsize=30,fontweight = 'bold')
plt.yticks(fontsize=30,fontweight = 'bold')
plt.xlabel('gamma',fontsize=35,fontweight = 'bold')
plt.ylabel('MIPGap difference',fontsize=35,labelpad = 20,fontweight = 'bold')

### Cut characteristic table

In [26]:
# Draw the cut characteristic table 
df = pd.DataFrame(columns = [str(gamma) for gamma in gamma_set
                            ])

avg_support_SSR, avg_zero_SSR = [],[]
num_no_support_SSR, num_no_zero_SSR = [],[]

for i in range(len(gamma_set)):
    avg_support_SSR.append(np.mean([len(j) for j in all_support_SSR[i]]))
    avg_zero_SSR.append(np.mean([len(j) for j in all_zero_SSR[i]]))
    num_no_support_SSR.append(len([j for j in all_support_SSR[i] if j == []]))
    num_no_zero_SSR.append(len([j for j in all_zero_SSR[i] if j == []]))

avg_num_inc,avg_num_exc = [],[]
avg_len_inc,avg_len_exc = [],[]
num_no_inc,num_no_exc = [],[]

for i in range(len(gamma_set)):
    avg_num_inc.append(np.mean(
                            [len(j) for j in all_inc_SCG[i] if j!=[[]] ] + \
                                [0 for j in all_inc_SCG[i] if j==[[]] ])
                                  )
    avg_num_exc.append(np.mean(
                            [len(j) for j in all_exc_SCG[i] if j!=[[]] ] + \
                                [0 for j in all_exc_SCG[i] if j==[[]] ])
                                  )
    fix_gamma_no_inc, fix_gamma_no_exc = 0,0
    fix_gamma_inc_len, fix_gamma_exc_len = [],[]
    for j in range(total_instance):
        inc_set , exc_set = all_inc_SCG[i][j] , all_exc_SCG[i][j]
        if inc_set == [[]]:
            fix_gamma_no_inc += 1
        else:
            fix_gamma_inc_len.append(np.mean([len(cut) for cut in inc_set]))
        if exc_set == [[]]:
            fix_gamma_no_exc += 1
        else:
            fix_gamma_exc_len.append(np.mean([len(cut) for cut in exc_set]))
            
    num_no_inc.append(fix_gamma_no_inc)
    num_no_exc.append(fix_gamma_no_exc)

    avg_len_inc.append(np.mean(fix_gamma_inc_len))
    avg_len_exc.append(np.mean(fix_gamma_exc_len))


# first add SSR
df.loc[len(df)] = avg_support_SSR
df.loc[len(df)] = avg_zero_SSR
df.loc[len(df)] = num_no_support_SSR
df.loc[len(df)] = num_no_zero_SSR

# next add SCG
df.loc[len(df)] = avg_num_inc
df.loc[len(df)] = avg_num_exc
df.loc[len(df)] = avg_len_inc
df.loc[len(df)] = avg_len_exc
df.loc[len(df)] = num_no_inc
df.loc[len(df)] = num_no_exc

    

df.index = ['average support that can be fixed (SSR)','average zero that can be fixed (SSR)',
            'num of no support fixed instances (SSR)','num of no zero fixed instances (SSR)',
            'average num of inc cuts (SCG)','average num of exc cuts (SCG)',
            'average len of inc cut (SCG)','average len of exc cut (SCG)',
            'num of no inc cut instances (SCG)','num of no exc cut instances (SCG)']

df

,3,4,5
average support that can be fixed (SSR),6.200000,6.000000,7.200000
average zero that can be fixed (SSR),83.800000,85.000000,85.000000
num of no support fixed instances (SSR),0.000000,0.000000,0.000000
num of no zero fixed instances (SSR),0.000000,0.000000,0.000000
average num of inc cuts (SCG),6.400000,6.000000,7.400000
average num of exc cuts (SCG),86.600000,87.400000,88.400000
average len of inc cut (SCG),1.028571,1.000000,1.040000
average len of exc cut (SCG),1.056349,1.038314,1.044904
num of no inc cut instances (SCG),0.000000,0.000000,0.000000
num of no exc cut instances (SCG),0.000000,0.000000,0.000000
